# Q1 Use the titanic data set, perform preprocessing by deal with missing values, drop irrelevant attributes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics

In [15]:
df = pd.read_csv('titanic_dataset.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [16]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [17]:
df["Age"] = df["Age"].fillna(df["Age"].median())

In [18]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [19]:
df = df.drop(columns = ["PassengerId", "Ticket", "Cabin", "Name"])

In [20]:
df = df.dropna()

In [21]:
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [22]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,28.0,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


# Q2 Use the scikit learn pipelines to perform the preprocessing - standardizing, encoding and model fitting in one step. 

In [23]:
df.dtypes

Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object

In [24]:
y = df["Survived"]
x = df
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2)

In [25]:
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

numerical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

In [26]:
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns
categorical_features = x.select_dtypes(include=['object']).columns


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numeric_features),
        ('cat', categorical_pipeline, categorical_features)])

In [27]:
from sklearn.naive_bayes import GaussianNB

gnb = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', GaussianNB())])

gnb.fit(train_x, train_y)
pred_y = gnb.predict(test_x)
metrics.accuracy_score(test_y, pred_y)

1.0

In [28]:
numeric_features

Index(['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

# Q3 Perform Bayes classification using cross validation.

In [36]:
from sklearn.model_selection import KFold, cross_val_score
k_fold = KFold(n_splits=9, shuffle=False, random_state=None)
clf = GaussianNB()
gnb = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', GaussianNB())])

scores = cross_val_score(gnb, x, y, cv=k_fold, n_jobs=1)

In [37]:
scores

array([1., 1., 1., 1., 1., 1., 1., 1., 1.])

# Q4 Tabulate using relevant measures of accuracy , Sensitivity and specificity.

In [ ]:
cf_matrix = metrics.confusion_matrix(test_y, pred_y)
plt.figure(figsize=(10,5))
display = metrics.ConfusionMatrixDisplay(confusion_matrix = cf_matrix, display_labels = gnb.classes_)
display.plot()
plt.show()

# Q5 Visualize the ROC curve and comment on performance

In [ ]:
pred_proba_y = gnb.predict_proba(test_x)
fpr, tpr, _ = sklearn.metrics.roc_curve(test_y,  pred_proba_y[::,1])
print(sklearn.metrics.roc_auc_score(test_y, pred_proba_y[::,1]))
print("The classifier has a very high AUC score which can be considered as an excellent analysis for the classification of diabetes ")
plt.plot(fpr,tpr, color = 'green')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()